In [2]:
import numpy as np
import pandas as pd

import tarfile
import gzip
import re
import os
import datetime as dt

In [3]:
year_num = 20 # Number of past years to consider
extremes_num = 10 # Number of hottest and coldest places to display

In [4]:
yearfiles = os.listdir("./gsod_all_years")
yearfiles.sort()
yearfiles = yearfiles[-year_num:]
years = [int(re.findall('\d+',yearfile)[0]) for yearfile in yearfiles]

In [5]:
station_loc = pd.read_csv('isd-history.csv')
station_loc = station_loc.replace([0.0, -999.0, -999.9],np.nan)
station_loc = station_loc[pd.notnull(station_loc['LAT']) & pd.notnull(station_loc['LON'])]
station_loc = station_loc[[int(re.findall('^\d{4}', str(end_year))[0])==max(years) for end_year in station_loc['END']]]
station_loc = station_loc[[int(re.findall('^\d{4}', str(beg_year))[0])<=min(years) for beg_year in station_loc['BEGIN']]]

In [6]:
station_loc['LBL'] = station_loc[['STATION NAME','STATE','CTRY']].apply(lambda x: x.str.cat(sep=', '), axis=1)
station_loc['ELEV_LBL'] = station_loc['ELEV(M)'].apply(lambda x: 'Elevation: '+str(x)+' m' if ~np.isnan(x) else np.nan)
station_loc['LBL'] = station_loc[['LBL','ELEV_LBL']].apply(lambda x: x.str.cat(sep='<br>'), axis=1)
station_loc = station_loc.drop(['STATION NAME','STATE','ELEV_LBL','ICAO','BEGIN','END'], axis=1)
#station_loc = station_loc.sample(stat_num)

In [7]:
df = pd.DataFrame([])
df_day = pd.DataFrame([])

def preprocess_station_file_content(content):
    headers=content.pop(0)
    headers=[headers[ind] for ind in [0,1,2,3,4,8,11,12]]
    for d in range(len(content)):
        content[d]=[content[d][ind] for ind in [0,1,2,3,5,13,17,18]]
    content=pd.DataFrame(content, columns=headers)
    content.rename(columns={'STN---': 'USAF'}, inplace=True)
    content['MAX'] = content['MAX'].apply(lambda x: re.sub("\*$","",x))
    content['MIN'] = content['MIN'].apply(lambda x: re.sub("\*$","",x))
    content[['WBAN','TEMP','DEWP','WDSP','MAX','MIN']] = content[['WBAN','TEMP','DEWP','WDSP','MAX','MIN']].apply(pd.to_numeric)
    content['YEARMODA']=pd.to_datetime(content['YEARMODA'], format='%Y%m%d', errors='ignore')
    content['YEAR']=pd.DatetimeIndex(content['YEARMODA']).year
    content['MONTH']=pd.DatetimeIndex(content['YEARMODA']).month
    content['DAY']=pd.DatetimeIndex(content['YEARMODA']).day
    return content

In [8]:
yearfile = yearfiles[-1]
print(yearfile)
i=0
tar = tarfile.open("./gsod_all_years/"+yearfile, "r")
print(len(tar.getmembers()[1:]))
#for member in np.random.choice(tar.getmembers()[1:], size=stat_num, replace=False):
for member in tar.getmembers()[1:]:
    name_parts = re.sub("\.op\.gz$","",re.sub("^\./","",member.name)).split("-")
    usaf = name_parts[0]
    wban = int(name_parts[1])
    if station_loc[(station_loc['USAF']==usaf) & (station_loc['WBAN']==wban)].shape[0]!=0:
        i=i+1
        #if i%(stat_num//10) == 0: print(i)
        f=tar.extractfile(member)
        f=gzip.open(f, 'rb')
        content=[re.sub(" +", ",", line.decode("utf-8")).split(",") for line in f.readlines()]
        content=preprocess_station_file_content(content)
        df_day = df_day.append(content[content['YEARMODA']==content['YEARMODA'].max()])
        content = content.groupby(['USAF','WBAN','YEAR','MONTH']).agg('median').reset_index()
        df = df.append(content)
tar.close()

gsod_2019.tar
11738


In [9]:
df

,USAF,WBAN,YEAR,MONTH,TEMP,DEWP,WDSP,MAX,MIN,DAY
0,010030,99999,2019,1,16.00,8.00,12.90,20.80,12.40,16.0
1,010030,99999,2019,2,18.10,9.65,12.75,21.85,13.20,14.5
2,010030,99999,2019,3,9.40,2.70,12.70,14.00,6.10,16.0
3,010030,99999,2019,4,24.65,19.75,8.70,30.75,18.25,8.5
0,719330,99999,2019,1,-2.50,-9.40,5.60,28.20,-11.20,16.0
...,...,...,...,...,...,...,...,...,...,...
3,719077,99999,2019,4,0.30,-3.70,7.70,9.50,-7.60,8.5
0,122350,99999,2019,1,30.50,28.60,7.60,34.00,27.90,16.0
1,122350,99999,2019,2,37.65,33.20,8.00,42.55,30.80,14.5
2,122350,99999,2019,3,40.40,34.10,8.30,45.90,34.50,16.0


In [10]:
station_loc

,USAF,WBAN,CTRY,LAT,LON,ELEV(M),LBL
11,010010,99999,NO,70.933,-8.667,9.0,"JAN MAYEN(NOR-NAVY), NO<br>Elevation: 9.0 m"
13,010014,99999,NO,59.792,5.341,48.8,"SORSTOKKEN, NO<br>Elevation: 48.8 m"
17,010020,99999,NO,80.050,16.250,8.0,"VERLEGENHUKEN, NO<br>Elevation: 8.0 m"
18,010030,99999,NO,77.000,15.500,12.0,"HORNSUND, NO<br>Elevation: 12.0 m"
21,010060,99999,NO,78.250,22.817,14.0,"EDGEOYA, NO<br>Elevation: 14.0 m"
...,...,...,...,...,...,...,...
29435,999999,53877,US,35.495,-82.614,655.6,"ASHEVILLE 8 SSW, NC, US<br>Elevation: 655.6 m"
29436,999999,53878,US,35.419,-82.557,641.0,"ASHEVILLE 13 S, NC, US<br>Elevation: 641.0 m"
29595,999999,93804,US,34.917,-81.950,244.1,"SPARTANBURG, SC, US<br>Elevation: 244.1 m"
29603,999999,93816,US,39.070,-88.533,178.9,"EFFINGHAM COUNTY MEMORIAL ARPT, IL, US<br>Elev..."


In [17]:
merged_df = pd.merge(df, station_loc, how="right", on="USAF")
merged_df

,USAF,WBAN_x,YEAR,MONTH,TEMP,DEWP,WDSP,MAX,MIN,DAY,WBAN_y,CTRY,LAT,LON,ELEV(M),LBL
0,010010,99999.0,2019.0,1.0,26.00,19.90,16.90,30.40,19.8,16.0,99999,NO,70.933,-8.667,9.0,"JAN MAYEN(NOR-NAVY), NO<br>Elevation: 9.0 m"
1,010010,99999.0,2019.0,2.0,26.85,22.85,15.90,31.45,23.0,14.5,99999,NO,70.933,-8.667,9.0,"JAN MAYEN(NOR-NAVY), NO<br>Elevation: 9.0 m"
2,010010,99999.0,2019.0,3.0,22.50,15.40,13.20,28.00,18.1,16.0,99999,NO,70.933,-8.667,9.0,"JAN MAYEN(NOR-NAVY), NO<br>Elevation: 9.0 m"
3,010010,99999.0,2019.0,4.0,32.35,28.25,11.15,34.20,29.6,8.5,99999,NO,70.933,-8.667,9.0,"JAN MAYEN(NOR-NAVY), NO<br>Elevation: 9.0 m"
4,010014,99999.0,2019.0,1.0,36.50,30.40,11.10,39.20,33.8,16.0,99999,NO,59.792,5.341,48.8,"SORSTOKKEN, NO<br>Elevation: 48.8 m"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31430,999999,93816.0,2019.0,4.0,54.90,42.80,5.90,70.00,47.7,9.0,94290,US,47.687,-122.255,18.3,"WSFO SEATTLE SAND POINT, WA, US<br>Elevation: ..."
31431,999999,53877.0,2019.0,1.0,35.60,9999.90,999.90,46.00,29.1,16.0,94290,US,47.687,-122.255,18.3,"WSFO SEATTLE SAND POINT, WA, US<br>Elevation: ..."
31432,999999,53877.0,2019.0,2.0,43.95,9999.90,999.90,59.00,34.8,14.5,94290,US,47.687,-122.255,18.3,"WSFO SEATTLE SAND POINT, WA, US<br>Elevation: ..."
31433,999999,53877.0,2019.0,3.0,44.60,9999.90,999.90,59.70,33.8,16.0,94290,US,47.687,-122.255,18.3,"WSFO SEATTLE SAND POINT, WA, US<br>Elevation: ..."


In [18]:
merged_clean_df = merged_df.loc[merged_df["USAF"] != "999999", :]
merged_clean_df

,USAF,WBAN_x,YEAR,MONTH,TEMP,DEWP,WDSP,MAX,MIN,DAY,WBAN_y,CTRY,LAT,LON,ELEV(M),LBL
0,010010,99999.0,2019.0,1.0,26.00,19.90,16.90,30.40,19.8,16.0,99999,NO,70.933,-8.667,9.0,"JAN MAYEN(NOR-NAVY), NO<br>Elevation: 9.0 m"
1,010010,99999.0,2019.0,2.0,26.85,22.85,15.90,31.45,23.0,14.5,99999,NO,70.933,-8.667,9.0,"JAN MAYEN(NOR-NAVY), NO<br>Elevation: 9.0 m"
2,010010,99999.0,2019.0,3.0,22.50,15.40,13.20,28.00,18.1,16.0,99999,NO,70.933,-8.667,9.0,"JAN MAYEN(NOR-NAVY), NO<br>Elevation: 9.0 m"
3,010010,99999.0,2019.0,4.0,32.35,28.25,11.15,34.20,29.6,8.5,99999,NO,70.933,-8.667,9.0,"JAN MAYEN(NOR-NAVY), NO<br>Elevation: 9.0 m"
4,010014,99999.0,2019.0,1.0,36.50,30.40,11.10,39.20,33.8,16.0,99999,NO,59.792,5.341,48.8,"SORSTOKKEN, NO<br>Elevation: 48.8 m"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30898,988360,99999.0,2019.0,4.0,83.10,75.45,5.30,92.20,75.2,8.5,99999,RP,6.922,122.060,10.1,"ZAMBOANGA INTL, RP<br>Elevation: 10.1 m"
30899,988510,41231.0,2019.0,1.0,83.80,69.40,7.70,90.50,73.4,16.0,41231,RP,14.800,120.267,18.3,"CUBI POINT NAVAL AIR STATION, RP<br>Elevation:..."
30900,988510,41231.0,2019.0,2.0,85.30,66.05,8.05,93.40,71.4,14.5,41231,RP,14.800,120.267,18.3,"CUBI POINT NAVAL AIR STATION, RP<br>Elevation:..."
30901,988510,41231.0,2019.0,3.0,85.90,69.10,6.40,95.00,72.7,16.0,41231,RP,14.800,120.267,18.3,"CUBI POINT NAVAL AIR STATION, RP<br>Elevation:..."
